Parallel Map on Files
------------------------

For each of a set of filenames, we parse JSON data contents, load that data into a Pandas DataFrame, and then output the result to another file with a nicer format, HDF5.

We find that parsing JSON is slow and so we parallelize the process using the [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) module to do this work in multiple processes.

### Objectives

*  Profile code to find bottleneck
*  Use `concurrent.futures` to `map` a function across many inputs in parallel


### Requirements

*  Pandas
*  concurrent.futures (standard in Python 3, `pip install futures` in Python 2)
*  snakeviz (for profile visualization, `pip install snakeviz`)


    pip install snakeviz
    pip install futures
    
### Extra Exercise

Try out alternative binary formats.  Perhaps try [feather](https://github.com/wesm/feather).

In [ ]:
%load_ext snakeviz

In [ ]:
from glob import glob
import json
import pandas as pd

In [ ]:
%%snakeviz

filenames = sorted(glob('../data/data-*.json'))

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    df.to_hdf(fn.replace('json', 'h5'), '/data')

Parallel Execution
--------------------

We can process each file independently and in parallel.  To accomplish this we'll transform the body of our for loop into a function and then use the [concurrent.futures.ProcessPoolExecutor](https://docs.python.org/3/library/concurrent.futures.html#executor-objects) to apply that function across all of the filenames in parallel using multiple processes.

### Before

Whenever we have code like the following:

```python
results = []
for x in L:
    results.append(f(x))
```

### After

We can instead write it as the following

```python
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

results = list(e.map(f, L))
```

In [ ]:
def load_parse_store(fn):
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    df.to_hdf(fn.replace('json', 'h5'), '/data')

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

list(e.map(load_parse_store, filenames))

Parallelism isn't everything
--------------------------------

We get a moderate increase in performance when using multiple processes.  However parallelism isn't the only way to accelerate this computation.  Recall that the bulk of the cost comes from the `json.loads` function.  A quick internet search on "fast json parsing in python" yields the [ujson](https://pypi.python.org/pypi/ujson) library as the top hit.

Knowing about and importing the optimized `ujson` library is just as effective as multi-core execution.

In [ ]:
import ujson as json

In [ ]:
%%time
filenames = sorted(glob('../data/data-*.json'))

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    df.to_hdf(fn.replace('json', 'h5'), '/data')

Of course, we can pair `ujson` with parallelism for even greater effect.

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

list(e.map(load_parse_store, filenames))

History: multiprocessing.Pool
--------------------------------

Perviously people have done multi-processing computations with the `multiprocessing.Pool` object, which behaves more or less identically.

However, today most library designers are coordinating around the `concurrent.futures` interface, so it's wise to move over.

In [ ]:
%%time 

from multiprocessing import Pool
p = Pool()

list(p.map(load_parse_store, filenames))

Conclusion
-----------

*  Used `snakeviz` to profile code
*  Used `concurrent.futures.ProcessPoolExecutor` for simple parallelism across many files
    *  Gained some speed boost (but not as much as expected)
    *  Lost ability to diagnose performance within parallel code
*  Saw that other options than parallelism exist to speed up code, including the `ujson` library.